In [2]:
# imports
import numpy as np
import pandas as pd
from datetime import datetime


In [56]:
# define date parser
d_parser = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

# read csv file (set date column as index)
df = pd.read_csv(
    './data/energy-charts_Stromproduktion_und_Börsenstrompreise_in_Deutschland_2020_Excel.csv',
    delimiter=';', parse_dates=['Datum (MT+1)'], date_parser=d_parser
    # index_col='Datum (MT+1)'
)

In [40]:
# copy df to edit df
energy_df = df.copy()

# rename date column
energy_df.rename(columns={'Datum (MT+1)': 'Datetime', 'Day Ahead Auktion (DE-LU)': 'Day Ahead Auktion/MWh' }, inplace=True)

In [43]:
# add Date column
energy_df['Date'] = energy_df['Datetime'].dt.date

In [46]:
# add weekday column
energy_df['Weekday'] = energy_df['Datetime'].dt.day_name()

In [48]:
# remove import saldo column, as all values are NaN
energy_df = energy_df.dropna(axis=1, how="all")

In [50]:
# Check if there is any na value in df
energy_df.replace('na', np.nan, inplace=True)
energy_df.replace('NA', np.nan, inplace=True)
energy_df.replace('Missing', np.nan, inplace=True)
print(energy_df.isna().value_counts())

Datetime  Kernenergie  Nicht Erneuerbar  Erneuerbar  Day Ahead Auktion/MWh  Date   Weekday
False     False        False             False       False                  False  False      35136
dtype: int64


In [51]:
# drop duplicates
energy_df.drop_duplicates(inplace=True)

In [52]:
# set Date column as index
energy_df.set_index('Datetime', inplace=True)


In [26]:
energy_df.head(10)

,Kernenergie,Nicht Erneuerbar,Erneuerbar,Day Ahead Auktion/MWh,Date,Weekday
Datetime,,,,,,
2020-01-01 00:00:00,8089.585,29700.989,14524.769,38.60,2020-01-01,Wednesday
2020-01-01 00:15:00,8087.935,29597.398,14407.463,38.60,2020-01-01,Wednesday
2020-01-01 00:30:00,8088.635,29433.049,14537.378,38.60,2020-01-01,Wednesday
2020-01-01 00:45:00,8086.715,29319.558,14695.598,38.60,2020-01-01,Wednesday
2020-01-01 01:00:00,8093.394,29202.803,14902.589,36.55,2020-01-01,Wednesday
2020-01-01 01:15:00,8090.135,29144.575,14898.388,36.55,2020-01-01,Wednesday
2020-01-01 01:30:00,8076.207,29123.509,14975.325,36.55,2020-01-01,Wednesday
2020-01-01 01:45:00,8064.899,29243.764,15021.521,36.55,2020-01-01,Wednesday
2020-01-01 02:00:00,8080.356,29402.351,15004.099,32.32,2020-01-01,Wednesday


In [5]:
energy_df.describe()
# what does it mean when the price is minus?

,Kernenergie,Nicht Erneuerbar,Erneuerbar,Day Ahead Auktion/MWh
count,35136.000000,35136.000000,35136.000000,35136.000000
mean,6934.673246,33765.069651,27962.757734,30.471738
std,969.703507,11159.545300,12106.069379,17.501029
min,3739.551000,11985.471000,7809.697000,-83.940000
25%,6450.204500,25593.516000,18140.908750,21.750000
50%,6793.422000,32771.112000,26610.473500,30.990000
75%,7834.915000,39948.887250,36752.224000,40.250000
max,8185.779000,68966.183000,69026.970000,200.040000


In [6]:
energy_df.corr()

/var/folders/cz/cwy65nds1w7_vfh7mtxwrks40000gn/T/ipykernel_3200/1385592123.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  energy_df.corr()


,Kernenergie,Nicht Erneuerbar,Erneuerbar,Day Ahead Auktion/MWh
Kernenergie,1.000000,0.407680,-0.135737,0.372776
Nicht Erneuerbar,0.407680,1.000000,-0.459684,0.796859
Erneuerbar,-0.135737,-0.459684,1.000000,-0.512600
Day Ahead Auktion/MWh,0.372776,0.796859,-0.512600,1.000000


In [11]:
# Analyzation - Weekday and Energy price

# Name dataframe grouped by weekeday
energy_df_grp_weekday = energy_df.groupby(['Weekday'])

In [16]:
# describe
energy_df_grp_weekday.describe()


Kernenergie                                                 \
                count         mean          std       min        25%   
Weekday                                                                
Friday         4992.0  6999.873143   917.873387  3851.843  6483.2795   
Monday         4992.0  6935.707435   996.903073  4434.400  6458.6635   
Saturday       4992.0  6828.283185  1003.081146  3901.165  6408.3365   
Sunday         4992.0  6669.734660  1074.507358  3739.551  6130.7430   
Thursday       5088.0  7073.832397   901.565573  4955.996  6472.7640   
Tuesday        4992.0  6969.493760   911.582489  4209.786  6498.4815   
Wednesday      5088.0  7060.688629   907.918966  4943.938  6492.5975   

                                           Nicht Erneuerbar                \
                 50%         75%       max            count          mean   
Weekday                                                                     
Friday     7002.2385  7815.59875  8168.552           4992.0  35594.819564   
Monday     6792.1620  7867.53000  8148.085           4992.0  35435.496522   
Saturday   6710.7600  7782.48400  8156.574           4992.0  28148.655849   
Sunday     6597.8485  7779.20400  8162.843           4992.0  25755.807607   
Thursday   7328.9660  7875.62150  8185.779           5088.0  37045.086670   
Tuesday    6828.9215  7809.49400  8167.582           4992.0  36886.230332   
Wednesday  7345.4490  7875.33900  8174.211           5088.0  37357.233690   

           ...   Erneuerbar            Day Ahead Auktion/MWh             \
           ...          75%        max                 count       mean   
Weekday    ...                                                            
Friday     ...  35767.08025  66538.478                4992.0  33.382268   
Monday     ...  35265.18025  67731.410                4992.0  32.391106   
Saturday   ...  36623.93650  61522.187                4992.0  25.210457   
Sunday     ...  38976.60450  61089.096                4992.0  18.136963   
Thursday   ...  35262.28850  66151.030                5088.0  35.097586   
Tuesday    ...  38347.49800  68055.035                4992.0  33.767548   
Wednesday  ...  36934.32775  69026.970                5088.0  35.137555   

                                                               
                 std    min      25%     50%      75%     max  
Weekday                                                        
Friday     13.440421  -8.77  24.9200  33.595  41.2525  106.35  
Monday     18.979678 -78.15  22.6125  33.365  43.2925  200.04  
Saturday   12.838657 -11.16  17.1250  26.260  34.0000   58.00  
Sunday     20.075361 -74.97   7.5700  21.445  32.0100   58.97  
Thursday   14.358696  -1.58  25.7650  34.040  43.1850  110.13  
Tuesday    18.152121 -83.94  24.2775  33.000  42.4600  189.25  
Wednesday  15.999207 -29.98  25.5375  33.785  43.2300  114.00  

[7 rows x 32 columns]

In [17]:
# Nuclear energie
energy_df_grp_weekday.describe().loc[:, 'Kernenergie']

,count,mean,std,min,25%,50%,75%,max
Weekday,,,,,,,,
Friday,4992.0,6999.873143,917.873387,3851.843,6483.2795,7002.2385,7815.59875,8168.552
Monday,4992.0,6935.707435,996.903073,4434.400,6458.6635,6792.1620,7867.53000,8148.085
Saturday,4992.0,6828.283185,1003.081146,3901.165,6408.3365,6710.7600,7782.48400,8156.574
Sunday,4992.0,6669.734660,1074.507358,3739.551,6130.7430,6597.8485,7779.20400,8162.843
Thursday,5088.0,7073.832397,901.565573,4955.996,6472.7640,7328.9660,7875.62150,8185.779
Tuesday,4992.0,6969.493760,911.582489,4209.786,6498.4815,6828.9215,7809.49400,8167.582
Wednesday,5088.0,7060.688629,907.918966,4943.938,6492.5975,7345.4490,7875.33900,8174.211


In [18]:
#  Non-renewable energy
energy_df_grp_weekday.describe().loc[:, 'Nicht Erneuerbar']

,count,mean,std,min,25%,50%,75%,max
Weekday,,,,,,,,
Friday,4992.0,35594.819564,10566.998275,15862.573,27803.94975,34211.2325,41151.10100,66045.204
Monday,4992.0,35435.496522,11339.662256,14290.529,27355.99500,34387.8780,42439.93275,63925.274
Saturday,4992.0,28148.655849,9210.785380,12709.939,21070.26650,26662.8965,32195.05900,56490.021
Sunday,4992.0,25755.807607,8835.164267,11985.471,18798.94700,23134.6945,30231.98100,54225.377
Thursday,5088.0,37045.086670,10161.526475,17328.133,30377.23050,35763.7220,41646.24200,67802.502
Tuesday,4992.0,36886.230332,10682.694580,15171.515,29346.11650,35628.9700,42127.36550,66942.701
Wednesday,5088.0,37357.233690,10780.798080,16312.197,30603.36000,35682.9530,41667.04125,68966.183


In [19]:
#  Renewable energy
energy_df_grp_weekday.describe().loc[:, 'Erneuerbar']

,count,mean,std,min,25%,50%,75%,max
Weekday,,,,,,,,
Friday,4992.0,27322.798680,11237.649553,8169.209,17909.19125,25852.3520,35767.08025,66538.478
Monday,4992.0,27547.043516,12030.721105,8525.884,17800.15975,26674.5865,35265.18025,67731.410
Saturday,4992.0,28379.580708,11776.160484,8695.429,18327.01300,27673.7515,36623.93650,61522.187
Sunday,4992.0,29231.175187,12489.182587,9096.588,19023.62850,27541.9440,38976.60450,61089.096
Thursday,5088.0,27303.590450,11279.358643,9001.541,18575.18400,26593.9185,35262.28850,66151.030
Tuesday,4992.0,28306.061736,13171.281288,7809.697,16859.05200,26625.2955,38347.49800,68055.035
Wednesday,5088.0,27667.409923,12523.821136,7810.523,18001.10325,25963.3425,36934.32775,69026.970


In [20]:
#  Energy price
energy_df_grp_weekday.describe().loc[:, 'Day Ahead Auktion/MWh']

,count,mean,std,min,25%,50%,75%,max
Weekday,,,,,,,,
Friday,4992.0,33.382268,13.440421,-8.77,24.9200,33.595,41.2525,106.35
Monday,4992.0,32.391106,18.979678,-78.15,22.6125,33.365,43.2925,200.04
Saturday,4992.0,25.210457,12.838657,-11.16,17.1250,26.260,34.0000,58.00
Sunday,4992.0,18.136963,20.075361,-74.97,7.5700,21.445,32.0100,58.97
Thursday,5088.0,35.097586,14.358696,-1.58,25.7650,34.040,43.1850,110.13
Tuesday,4992.0,33.767548,18.152121,-83.94,24.2775,33.000,42.4600,189.25
Wednesday,5088.0,35.137555,15.999207,-29.98,25.5375,33.785,43.2300,114.00
